# Recommendation Script
## Customers who bought albums from this artist also purchased from the following artists
### Upload a list of 'target' artist and get the top 10 recommende artists for each
#### Use case:  The merchansizer of a  music flash deal site needs select upsell products for  the daily customer emails. She his planning out next weeks emails and needs to select recommendations for each offer.

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

## Import list of target artists (artists for which you'd like recommendations)

In [2]:
target_artists = pd.read_csv('~/Google Drive/DS_Projects/Artist_Recommendation/data/pm_target_artists.csv')
target_artists = target_artists[['Artist']]

In [3]:
target_artists

,Artist
0,Miles Davis
1,Radiohead
2,Bob Dylan
3,Steve Earl
4,The Cure


## Import Products and LineItems data and join on "Email"

In [4]:
products = pd.read_csv('~/Google Drive/DS_Projects/Artist_Recommendation/data/products_export.csv')
df_prod = pd.DataFrame(products, columns = ['UPC', 'Artist', 'Title'])

line_items = pd.read_csv('~/Google Drive/DS_Projects/Artist_Recommendation/data/pm_line_items_export.csv')
line_items_df = pd.DataFrame(line_items, columns = ['Email','Lineitem_sku'])
line_items_df.columns = ['Email', 'UPC']

lines = pd.merge(line_items_df, df_prod, on= 'UPC' )


/Users/steffen/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Group by Artist And Email and count number of rows ( = line items ordered)

In [5]:
usersGrp = lines.groupby([lines['Email'],lines['Artist']])
usersArt = usersGrp['UPC'].count()
usersArt = pd.DataFrame(usersArt).reset_index()

## Write function to find the 10 most likely artists a customer will purchase given he has purchased the target artist.

In [6]:
# write function to create list of top recommendation for ONE artist. Baysian: Given that a customer ordered on artist
# what's the probability they'll order any of the other artists?

def top10Rec(ArtistName):
    # filter for users that have ordered the 'Target' artist
    likeUsers = pd.DataFrame(usersArt['Email'][usersArt['Artist'] == ArtistName])
    
    # find all orders by these users
    likeUsersArtist = pd.merge(usersArt,likeUsers, on = 'Email', how = 'inner')
    
    # group by the artists they've ordered
    grUserByArtist = likeUsersArtist.groupby(likeUsersArtist['Artist'])

    # how many orders did these users place for each artist
    UsersByArtist = grUserByArtist.count()
    UsersByArtist = UsersByArtist[['Email']]
    UsersByArtist = UsersByArtist.reset_index()
    
    # Calculate the probability of them ordering that artist
    UsersByArtist['Prob'] = UsersByArtist['Email']/len(likeUsers)
    UsersByArtist = UsersByArtist.sort(['Email'],ascending = False)
    UsersByArtist['OriginalArtist'] = ArtistName
    top10 = UsersByArtist.iloc[1:11,:]
    return(top10)



In [7]:
top10Rec('Metallica')

/Users/steffen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Artist,Email,Prob,OriginalArtist
60,Black Sabbath,112,0.110236,Metallica
228,Iron Maiden,97,0.095472,Metallica
608,Various Artists,95,0.093504,Metallica
285,Led Zeppelin,86,0.084646,Metallica
7,AC/DC,70,0.068898,Metallica
380,Ozzy Osbourne,69,0.067913,Metallica
527,The Beatles,66,0.064961,Metallica
184,Foo Fighters,66,0.064961,Metallica
13,Aerosmith,64,0.062992,Metallica
130,Deftones,61,0.060039,Metallica


## Write function that take list of target artists as input and returns top 10 'likely to purchase' artists.

In [8]:
def recommend(ArtistList):
    recFinal = pd.DataFrame()
    # loop over artist list
    for Artist in ArtistList['Artist']:
        rec = top10Rec(Artist)
        recFinal = recFinal.append(rec)
        recFinal = recFinal[['OriginalArtist','Artist','Prob']]
        recFinal.to_csv('~/Google Drive/DS_Projects/Artist_Recommendation/output/pm_recommendations.csv')
    return recFinal 


In [9]:
test = recommend(target_artists)


/Users/steffen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [10]:
test

,OriginalArtist,Artist,Prob
208,Miles Davis,Jimi Hendrix,0.168421
51,Miles Davis,Bob Dylan,0.149123
240,Miles Davis,Led Zeppelin,0.140351
131,Miles Davis,Elvis Presley,0.121053
481,Miles Davis,The Rolling Stones,0.119298
518,Miles Davis,Various Artists,0.112281
66,Miles Davis,Buddy Guy,0.096491
445,Miles Davis,The Beatles,0.091228
42,Miles Davis,Black Sabbath,0.084211
327,Miles Davis,Paul McCartney,0.080702
